In [55]:
import time

class TicTacToeGame:
    def __init__(self):
        self.board = [' '] * 9
        self.champion = None
        self.total_time = 0 
        self.moves_count = 0  

    def display_board(self):
        print("\n    0   1   2")
        print("  +---+---+---+")
        for row in range(3):
            row_cells = [self.board[row * 3 + col] for col in range(3)]
            print(f"{row} | " + " | ".join(row_cells) + " |")
            print("  +---+---+---+")

    def get_available_moves(self):
        return [i for i in range(9) if self.board[i] == ' ']

    def has_empty_squares(self):
        return ' ' in self.board

    def place_move(self, index, symbol):
        if self.board[index] == ' ':
            self.board[index] = symbol
            if self.verify_winner(index, symbol):
                self.champion = symbol
            return True
        return False

    def verify_winner(self, index, symbol):
        row = index // 3
        col = index % 3

        # Check the row
        if all(self.board[row * 3 + i] == symbol for i in range(3)):
            return True
        # Check the column
        if all(self.board[col + 3 * i] == symbol for i in range(3)):
            return True
        # Check diagonals
        if index % 2 == 0:
            if all(self.board[i] == symbol for i in [0, 4, 8]) or all(self.board[i] == symbol for i in [2, 4, 6]):
                return True
        return False

    def alpha_beta_pruning(self, is_maximizing, player_symbol, computer_symbol, alpha, beta):
        # Check terminal states
        if self.champion == computer_symbol:
            return 1
        elif self.champion == player_symbol:
            return -1
        elif not self.has_empty_squares():
            return 0

        if is_maximizing:
            best_score = -float('inf')
            for move in self.get_available_moves():
                self.board[move] = computer_symbol
                previous_champion = self.champion
                if self.verify_winner(move, computer_symbol):
                    self.champion = computer_symbol
                score = self.alpha_beta_pruning(False, player_symbol, computer_symbol, alpha, beta)
                self.board[move] = ' '
                self.champion = previous_champion
                best_score = max(score, best_score)
                alpha = max(alpha, best_score)
                if beta <= alpha: 
                    break
            return best_score
        else:
            best_score = float('inf')
            for move in self.get_available_moves():
                self.board[move] = player_symbol
                previous_champion = self.champion
                if self.verify_winner(move, player_symbol):
                    self.champion = player_symbol
                score = self.alpha_beta_pruning(True, player_symbol, computer_symbol, alpha, beta)
                self.board[move] = ' '
                self.champion = previous_champion
                best_score = min(score, best_score)
                beta = min(beta, best_score)
                if beta <= alpha:  # Alpha cut-off
                    break
            return best_score

    def best_move(self, player_symbol, computer_symbol):
        best_score = -float('inf')
        move = None
        alpha = -float('inf')
        beta = float('inf')
        
        # Start measuring time for the algorithm's decision
        start_time = time.time()

        for possible_move in self.get_available_moves():
            self.board[possible_move] = computer_symbol
            previous_champion = self.champion
            if self.verify_winner(possible_move, computer_symbol):
                self.champion = computer_symbol
            score = self.alpha_beta_pruning(False, player_symbol, computer_symbol, alpha, beta)
            self.board[possible_move] = ' '
            self.champion = previous_champion
            if score > best_score:
                best_score = score
                move = possible_move
        
        # End measuring time for the algorithm's decision
        end_time = time.time()

        # Update the total time and count the move
        move_time = end_time - start_time
        self.total_time += move_time
        self.moves_count += 1

        # Print the time taken for the decision
        print(f"\nComputer took {move_time:.4f} seconds to decide on a move.")

        return move

    def total_time_taken(self):
        return self.total_time

def start_game():
    game = TicTacToeGame()

    print("\nWelcome to Tic Tac Toe!")
    print("\nGet ready to challenge the Computer!")
    game.display_board()

    player_symbol = ''
    while player_symbol not in ['X', 'O']:
        player_symbol = input("\nChoose your symbol (X or O): ").upper()
        if player_symbol not in ['X', 'O']:
            print("Invalid choice! Please pick either 'X' or 'O'.")

    computer_symbol = 'O' if player_symbol == 'X' else 'X'

    while game.has_empty_squares():
        # Player's turn
        move = None
        while move not in game.get_available_moves():
            try:
                user_input = input("\nEnter your move (row and column separated by space, e.g., '1 2'): ")
                row, col = map(int, user_input.split())
                move = row * 3 + col
                if move not in game.get_available_moves():
                    print("Invalid move! That spot is already taken.")
            except (ValueError, IndexError):
                print("Please enter valid row and column numbers (0, 1, or 2).")

        game.place_move(move, player_symbol)
        game.display_board()

        if game.champion:
            print(f"\nCongratulations! You ({player_symbol}) win!")
            return

        if not game.has_empty_squares():
            break

        # Computer's turn
        print("\nComputer is making a move...")

        move = game.best_move(player_symbol, computer_symbol)

        # Adding personality to the AI's moves
        if move in [0, 2, 6, 8]:  # After Taking a Corner
            print(f"\nI love the corners. I placed '{computer_symbol}' at position ({move // 3}, {move % 3})")
        elif move == 4:  # After Taking the Center
            print(f"\nI got the Center, that’s where the power lies! I placed '{computer_symbol}' at position (1, 1)")
        elif game.verify_winner(move, computer_symbol):  # When It's About to Win
            print(f"\nNice move!.I Placed '{computer_symbol}' at position ({move // 3}, {move % 3})")
        elif game.verify_winner(move, player_symbol):  # When It's Blocking the Player
            print(f"\nNice try, but I’m blocking that one! I placed '{computer_symbol}' at position ({move // 3}, {move % 3})")
        elif game.has_empty_squares():  # After a Defensive Move
            print(f"\nI’m always thinking one step ahead. I placed '{computer_symbol}' at position ({move // 3}, {move % 3})")
        else:  # When It's About to Lose (or Playing for a Tie)
            print(f"\nI’m in a tough position, but I’ll do my best! I placed '{computer_symbol}' at position ({move // 3}, {move % 3})")

        game.place_move(move, computer_symbol)
        game.display_board()

        if game.champion:
            print(f"\nComputer ({computer_symbol}) wins! Better luck next time!")
            return
            
    print("\nIt's a tie!")
    # After the game ends, print the total time taken by the AI
    print(f"\nTotal time taken by the Alpha_beta_pruning Algorithm to make moves: {game.total_time_taken():.4f} seconds")


if __name__ == '__main__':
    start_game()



Welcome to Tic Tac Toe!

Get ready to challenge the Computer!

    0   1   2
  +---+---+---+
0 |   |   |   |
  +---+---+---+
1 |   |   |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+



Choose your symbol (X or O):  x

Enter your move (row and column separated by space, e.g., '1 2'):  0 0



    0   1   2
  +---+---+---+
0 | X |   |   |
  +---+---+---+
1 |   |   |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+

Computer is making a move...

Computer took 0.0382 seconds to decide on a move.

I got the Center, that’s where the power lies! I placed 'O' at position (1, 1)

    0   1   2
  +---+---+---+
0 | X |   |   |
  +---+---+---+
1 |   | O |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+



Enter your move (row and column separated by space, e.g., '1 2'):  0 1



    0   1   2
  +---+---+---+
0 | X | X |   |
  +---+---+---+
1 |   | O |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+

Computer is making a move...

Computer took 0.0010 seconds to decide on a move.

I love the corners. I placed 'O' at position (0, 2)

    0   1   2
  +---+---+---+
0 | X | X | O |
  +---+---+---+
1 |   | O |   |
  +---+---+---+
2 |   |   |   |
  +---+---+---+



Enter your move (row and column separated by space, e.g., '1 2'):  2 0



    0   1   2
  +---+---+---+
0 | X | X | O |
  +---+---+---+
1 |   | O |   |
  +---+---+---+
2 | X |   |   |
  +---+---+---+

Computer is making a move...

Computer took 0.0000 seconds to decide on a move.

I’m always thinking one step ahead. I placed 'O' at position (1, 0)

    0   1   2
  +---+---+---+
0 | X | X | O |
  +---+---+---+
1 | O | O |   |
  +---+---+---+
2 | X |   |   |
  +---+---+---+



Enter your move (row and column separated by space, e.g., '1 2'):  2 1



    0   1   2
  +---+---+---+
0 | X | X | O |
  +---+---+---+
1 | O | O |   |
  +---+---+---+
2 | X | X |   |
  +---+---+---+

Computer is making a move...

Computer took 0.0000 seconds to decide on a move.

I’m always thinking one step ahead. I placed 'O' at position (1, 2)

    0   1   2
  +---+---+---+
0 | X | X | O |
  +---+---+---+
1 | O | O | O |
  +---+---+---+
2 | X | X |   |
  +---+---+---+

Computer (O) wins! Better luck next time!
